In [16]:
import pyrealsense2 as rs
import numpy as np
import cv2
from ultralytics import YOLO
import datetime
import socket
import math
import json

In [ ]:
model = YOLO("pingpang.pt")
BALL_RADIUS = 0.02  # 球半径，单位m
height=0.264
width=0.2935
z_=0.04
x_=0.48
y_=0.4# 机器臂参数
pipeline = rs.pipeline()  # 定义流程pipeline，创建一个管道
config = rs.config()  # 定义配置config
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 60)  # 配置depth流
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)  # 配置color流
# 创建对齐对象与color流对齐
align_to = rs.stream.color  # align_to 是计划对齐深度帧的流类型
align = rs.align(align_to)  # rs.align 执行深度帧与其他帧的对齐
def get_aligned_images():
    frames = pipeline.wait_for_frames()  # 等待获取图像帧，获取颜色和深度的框架集
    aligned_frames = align.process(frames)  # 获取对齐帧，将深度框与颜色框对齐

    aligned_depth_frame = aligned_frames.get_depth_frame()  # 获取对齐帧中的的depth帧
    aligned_color_frame = aligned_frames.get_color_frame()  # 获取对齐帧中的的color帧

    #### 获取相机参数 ####
    depth_intrin = (
        aligned_depth_frame.profile.as_video_stream_profile().intrinsics
    )  # 获取深度参数（像素坐标系转相机坐标系会用到）

    color_intrin = (
        aligned_color_frame.profile.as_video_stream_profile().intrinsics
    )  # 获取相机内参

    #### 将images转为numpy arrays ####
    img_color = np.asanyarray(aligned_color_frame.get_data())  # RGB图
    img_depth = np.asanyarray(aligned_depth_frame.get_data())  # 深度图（默认16位）

    return color_intrin, depth_intrin, img_color, img_depth, aligned_depth_frame
def get_3d_camera_coordinate(depth_pixel, aligned_depth_frame, depth_intrin):
    x = depth_pixel[0]
    y = depth_pixel[1]
    dis = aligned_depth_frame.get_distance(x, y)  # 获取该像素点对应的深度
    camera_coordinate = rs.rs2_deproject_pixel_to_point(depth_intrin, depth_pixel, dis+BALL_RADIUS)
    return dis, camera_coordinate


In [7]:
height=0.264
width=0.2935
[197,357],[0,0,0]
[360,350],[0.297,0,0]
[192,231],[0,0.23,0]
[354,224],[0.297,0.23,0]

([354, 224], [0.297, 0.23, 0])

In [9]:
check_size=20
# w h分别是棋盘格模板长边和短边规格（角点个数）
w = 9
h = 7
intrin = {
    "mtx": np.array([[607.961, 0.0, 324.761],
                     [0.0, 607.961, 252.966],
                     [0.0, 0.0, 1.0]],
                    dtype=np.float64),
    "dist": np.array([[0,0,0,0,0]], dtype=np.float64),
}
rvec_matrix_sum=np.array([[0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0]])
sum=0
pipe_profile = pipeline.start(config)  # streaming流开始
while True:
    color_intrin, depth_intrin, img, img_depth, aligned_depth_frame = (
                get_aligned_images()
            )
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # print(cv2.TERM_CRITERIA_EPS,'',cv2.TERM_CRITERIA_MAX_ITER)

    # 世界坐标系中的棋盘格点,例如(0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)，去掉Z坐标，记为二维矩阵，认为在棋盘格这个平面上Z=0
    objp = np.zeros((w * h, 3), np.float32)  # 构造0矩阵，80行3列，用于存放角点的世界坐标
    objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)  # 三维网格坐标划分

    # 储存棋盘格角点的世界坐标和图像坐标对
    objpoints = []  # 在世界坐标系中的三维点
    imgpoints = []  # 在图像平面的二维点

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 粗略找到棋盘格角点 这里找到的是这张图片中角点的亚像素点位置，共10*8 = 980个点，gray必须是8位灰度或者彩色图，（w,h）为角点规模
    ret, corners = cv2.findChessboardCorners(gray, (w, h))
    # 如果找到足够点对，将其存储起来
    if ret:
        # 精确找到角点坐标
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        worldpoint = objp * check_size  # 棋盘格的宽度
        imagepoint = np.squeeze(corners)  # 将corners降为二维

        (success, rvec, tvec) = cv2.solvePnP(worldpoint, imagepoint, intrin["mtx"], intrin["dist"])
        
        rvec_matrix = cv2.Rodrigues(rvec)[0]
        # 将角点在图像上显示
        cv2.drawChessboardCorners(img, (w, h), corners, ret)
        sum=sum+1
        rvec_matrix_sum=rvec_matrix_sum+rvec_matrix
    cv2.imshow('findCorners', img)
    if cv2.waitKey(1) & 0xFF == 27:
        break
# 停止流
pipeline.stop()
cv2.destroyAllWindows()
print(rvec_matrix_sum/sum)
camera_intrin={
    "R":(rvec_matrix_sum/sum).tolist()
}
with open("rotation.json","w") as f:
    json.dump(camera_intrin,f)
    print("加载入文件完成...")

[[    0.39934     0.01941  -0.0021175]
 [  -0.019426     0.39885   -0.048849]
 [-0.00014115    0.019631      0.9988]]
加载入文件完成...


In [15]:
WORLD2CAMERA=False
with open("rotation.json",'r') as file:
    dict=json.load(file)
R=np.array(dict["R"])
filename = (
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S").replace(":", "_") + ".txt"
)
with open(filename, "a") as file:
    pipe_profile = pipeline.start(config)  # streaming流开始
    while True:
        color_intrin, depth_intrin, img_color, img_depth, aligned_depth_frame = (
                    get_aligned_images()
                )
        results = model(img_color)
        if len(results[0].boxes)>0:
            result = results[0].boxes[0] # 访问检测结果
            x1, y1, x2, y2 = result.xyxy[0]
            confidence = result.conf[0]
            cls = result.cls[0]
            label = model.names[int(cls)]

            # 计算中心点

            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            img_color[int(y1.item())-1,int(x1.item())-1,:2]=0
            img_color[int(y2.item())-1,int(x1.item())-1,:2]=0
            img_color[int(y1.item())-1,int(x2.item())-1,:2]=0
            img_color[int(y2.item())-1,int(x2.item())-1,:2]=0

            # 获取中心点的三维坐标
            dis, camera_coordinate = get_3d_camera_coordinate(
                [center_x, center_y], aligned_depth_frame, depth_intrin
            )
            camera_coordinate=np.dot(camera_coordinate,R)
            while WORLD2CAMERA==False:
                orignepose=camera_coordinate
                WORLD2CAMERA=True
            print(camera_coordinate,orignepose)
            camera_coordinate=[camera_coordinate[i]-orignepose[i] for i in range(len(camera_coordinate))]
            print(camera_coordinate)
            camera_coordinate=[camera_coordinate[i]+[x_,y_,z_][i] for i in range(len(camera_coordinate))]

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
            file.write(
                f"{timestamp}, X: {camera_coordinate[0]:.2f}, Y: {camera_coordinate[1]:.2f}, Z: {camera_coordinate[2]:.2f}\n"
            )
        
        cv2.imshow("YOLO Detection", img_color)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    # 停止流
    pipeline.stop()
    cv2.destroyAllWindows()


0: 480x640 1 pingpang, 101.0ms
Speed: 1.4ms preprocess, 101.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
[  -0.042144    0.087026     0.62608] [  -0.042144    0.087026     0.62608]
[np.float64(0.0), np.float64(0.0), np.float64(0.0)]

0: 480x640 1 pingpang, 106.7ms
Speed: 8.4ms preprocess, 106.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
[  -0.042912    0.086849      0.6251] [  -0.042144    0.087026     0.62608]
[np.float64(-0.0007679205025002758), np.float64(-0.00017738776425722802), np.float64(-0.0009800324787424541)]

0: 480x640 1 pingpang, 59.2ms
Speed: 1.7ms preprocess, 59.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
[  -0.042542    0.086588     0.62614] [  -0.042144    0.087026     0.62608]
[np.float64(-0.00039741947629010066), np.float64(-0.0004377385924551336), np.float64(5.33396740032277e-05)]

0: 480x640 1 pingpang, 61.0ms
Speed: 1.4ms preprocess, 61.0ms inference, 0.7ms postprocess per image at shape (1,